In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

9.1

In [ ]:
trybik = cv2.imread('trybik.jpg')
trybik_gray = cv2.cvtColor(trybik, cv2.COLOR_BGR2GRAY)
trybik_binary = cv2.threshold(trybik_gray, 175, 255, cv2.THRESH_BINARY)[1]
trybik_binary = cv2.medianBlur(trybik_binary, 5)
trybik_binary = cv2.bitwise_not(trybik_binary)
plt.imshow(trybik_binary, cmap='gray')
plt.axis('off')
plt.show()
contours, hierarchy = cv2.findContours (trybik_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)


In [ ]:
trybik_copy = trybik.copy()
trybik_contours = cv2.drawContours(trybik_copy, contours, 0, (0, 255, 0))
plt.imshow(trybik_contours)
plt.axis('off')
plt.show()

In [ ]:
sobelx = cv2.Sobel(trybik_binary, cv2.CV_64F, 1, 0, ksize=5)
sobely = cv2.Sobel(trybik_binary, cv2.CV_64F, 0, 1, ksize=5)

amplitude = np.sqrt(sobelx**2 + sobely**2)
amplitude = amplitude/np.amax(amplitude)

orientation = np.rad2deg(np.arctan2(sobely, sobelx))
orientation = orientation + 180

moments = cv2.moments(trybik_binary)
center_x = int(moments['m10'] / moments['m00'])
center_y = int(moments['m01'] / moments['m00'])

In [ ]:
Rtable = [[] for _ in range(360)]

for contour in contours[0]:
    x = contour[0][0]
    y = contour[0][1]
    
    angle = orientation[y, x] % 360
    angle = int(angle)
    vector = np.array([x - center_x, y - center_y])
    length = np.sqrt(vector[0]**2 + vector[1]**2)
    vector_orientation = np.rad2deg(np.arctan2(vector[1], vector[0]))
    Rtable[angle].append((length, vector_orientation))

In [ ]:
trybiki2 = cv2.imread('trybiki2.jpg')
trybiki2_gray = cv2.cvtColor(trybiki2, cv2.COLOR_BGR2GRAY)
trybiki2_binary = cv2.threshold(trybiki2_gray, 92, 255, cv2.THRESH_BINARY)[1]
plt.imshow(trybiki2_binary, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
sobelx2 = cv2.Sobel(trybiki2_binary, cv2.CV_64F, 1, 0, ksize=5)
sobely2 = cv2.Sobel(trybiki2_binary, cv2.CV_64F, 0, 1, ksize=5)

amplitude2 = np.sqrt(sobelx2**2 + sobely2**2)
amplitude2 = amplitude2/np.amax(amplitude2)

orientation2 = np.rad2deg(np.arctan2(sobely2, sobelx2))
orientation2 = orientation2 + 180

In [ ]:
result = np.zeros(trybiki2_binary.shape[:2], dtype=np.uint16)

for x in range(amplitude2.shape[0]):
    for y in range(amplitude2.shape[1]):
        if amplitude2[x, y] > 0.5:
            angle = orientation2[x, y] % 360
            angle = angle.astype(np.uint16)
            column = Rtable[angle]

            for r, fi in column:
                x1 = int(x + r * np.sin(np.deg2rad(fi)))
                y1 = int(y + r * np.cos(np.deg2rad(fi)))
                if 0 <= x1 < result.shape[0] and 0 <= y1 < result.shape[1]:
                    result[x1, y1] += 1

result_ = np.where(result.max() * 0.5 < result)
result_

In [ ]:
plt.subplots(1,2, figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(np.log(result + 1), 'gray')

plt.subplot(1, 2, 2)
plt.imshow(trybiki2)
plt.show()

In [ ]:
for i in range(len(result_[0])):
    cv2.circle(trybiki2, (int(result_[1][i]), int(result_[0][i])), 2, (255, 0, 0))
    cv2.drawContours(trybiki2, contours, 0, (255, 0, 0), offset=(result_[1][i] - center_x, result_[0][i] - center_y))

In [ ]:
plt.imshow(trybiki2)
plt.show()